In [ ]:
# !pip install seaborn
# !pip install bokeh
# !pip install nltk
# !pip install wordcloud
# !pip install tqdm
# !pip install emoji


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
#import bokeh
import nltk
#import numpy as np
from nltk.tokenize import RegexpTokenizer
from wordcloud import WordCloud
import string
from os import path
from tqdm import tqdm
%matplotlib inline
import random

In [ ]:
nltk.download('punkt')
nltk.download("stopwords")

In [ ]:
DaysofWeeks=[ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')

In [ ]:
def check_date(date):
    X=dict()
    try:
        if '/' in date and ':' in date and ',' in date:
            X['Date']=date.split(',')[0]
            X['time']=date.split(',')[1][1:]
            X['Day']=int(X['Date'].split('/')[1])
            X['Month']=int(X['Date'].split('/')[0])
            X['Year']=int(X['Date'].split('/')[2])
            #X['Weekday']= DaysofWeeks[(datetime.datetime(X['Year'],X['Month'],X['Day']).weekday())]
            if date.split()[2]=='AM':
                X['hour']=int(X['time'].split(':')[0])
            elif date.split()[2]=='PM':
                X['hour']=int(X['time'].split(':')[0])+12
            X['mins']=int(X['time'].split(':')[1][:2])
            return X
    except Exception as e:
        X=dict()
    return X
    

In [ ]:
def get_sender(line):
    if ':' in line:
        if len(line.split(':')[0])<26:
            return line.split(':')[0]
    return ''

In [ ]:
col_list=['Month', 'Day', 'Year', 'Date', 'hour', 'mins', 'time', 'sender', 'text']
chat=pd.DataFrame(columns=col_list)

In [ ]:
num_messages=0
with open("<path to file>",'r',encoding="utf8") as file:
    line = file.readline().encode('utf-8').decode('utf-8-sig')
    while line:
    #for i in tqdm(range(42000)):
        try:
            date_split=line.split(" - ")
            #print(date_split)
            if len(date_split)>0:
                X=check_date(date_split[0])
                if len(X)>1:
                    date_split[1]
                    X['sender']=get_sender(date_split[1])
                    if len(X['sender']):
                        X['text']=date_split[1][len(X['sender'])+2:]
                    else:
                        X['text']=[date_split[1]]
                    #for key in X:
                    chat.loc[num_messages]=X
                    num_messages=num_messages+1
                else:
                    chat.iloc[-1]['text']+(line)
        except Exception as e:
            print (e)
        line = file.readline()

In [ ]:
chat.head(5)

TO PLOT EVERYTHING IN MATPLOTLIB: SEABORN: BOKEH

In [ ]:
df=pd.DataFrame(chat.groupby(['sender']).size().rename('counts'))
df=df.reset_index()
df=df.sample(frac=1).reset_index(drop=True)
filename="<include file name>"
df.plot(x='sender', y= 'counts',kind='bar',title=("Number of messages on "+filename+" : "+chat_start+"-"+chat_end), figsize=(15,10))
plt.savefig("count_bar.jpg")
plt.figure(figsize=(15,15))
patches, texts = plt.pie(
    # using data total)arrests
    df.counts,
    # with the labels being officer names
    labels=df.sender,
    # with no shadows
    shadow=False,
    # with the percent listed as a fraction
    )
# plt.legend(patches, F1.sender, bbox_to_anchor=(-0.1, 1.),
#            fontsize=18)
plt.tight_layout()
plt.savefig('piechart.jpg', bbox_inches='tight')

In [ ]:
#pandas in-built
chat_start= chat.iloc[0]['Date']
chat_end=chat.iloc[-1]['Date']
weeks=int((datetime.datetime(int(chat.iloc[-1]['Year']),int(chat.iloc[-1]['Month']),int(chat.iloc[-1]['Day'])) - datetime.datetime(int(chat.iloc[0]['Year']),int(chat.iloc[0]['Month']),int(chat.iloc[0]['Day']))).days/7)

In [ ]:
df=pd.DataFrame(chat.groupby(['Day','Month']).size().rename('counts'))
df=df.reset_index()

# fig=plt.figure(figsize=(20,10))
# ax1=fig.add_subplot(211)
gridkw = dict(height_ratios=[4, 1])
fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw=gridkw,figsize=(20,15)) 

result = df.pivot(index='Month', columns='Day', values='counts')
sns.heatmap(result, annot=True, fmt="g", cmap='viridis', ax=ax1, cbar_kws={"orientation": "horizontal"})
ax1.set_title("Monthly heatmap")
plt.title("Heatmap over the days of the month")
df=pd.DataFrame(chat.groupby(['Day']).size().rename('counts'))
df=df.reset_index()
#ax2=fig.add_subplot(212)
df.plot(x=['Day'], y= 'counts',kind='bar',title=("Group "+chat_start+"-"+chat_end),ax=ax2)
plt.savefig('monthly_map.jpg')
plt.show()


In [ ]:
chat['Weekday']= chat.apply(lambda X: (datetime.datetime(int(X['Year']),int(X['Month']),int(X['Day'])).weekday()), axis=1)

df=pd.DataFrame(chat.groupby(['Weekday']).size().rename('counts'))
df=df.reset_index()
df['counts']=df.counts.apply(lambda x: x/weeks)
fig = plt.figure(figsize=(15,5))
ax = plt.subplot(111)
plt.xticks(df.Weekday, DaysofWeeks)
ax.bar(df.Weekday, df.counts)
plt.title("Average message count for each day of the week")
plt.savefig("week_count.jpg")
plt.show()
#F4.plot(x='Weekday', y= 'counts',kind='bar',label=DaysofWeeks, title=("r/unitedwesend "+chat_start+"-"+chat_end),figsize=(15,10))

In [ ]:
df=pd.DataFrame(chat.groupby(['Weekday','hour']).size().rename('counts'))
df=df.reset_index()

# fig=plt.figure()
# ax1=fig.add_subplot(211)

gridkw = dict(height_ratios=[4, 1])
fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw=gridkw,figsize=(20,15)) 
#plt.yticks(F6.Weekday, DaysofWeeks)
result = df.pivot(index='Weekday', columns='hour', values='counts')
sns.heatmap(result, annot=True,yticklabels=DaysofWeeks, fmt="g", ax=ax1, cbar_kws={"orientation": "horizontal"})
ax1.set_title("weekly heatmap")
df=pd.DataFrame(chat.groupby(['hour']).size().rename('counts'))
df=df.reset_index()
#ax2=fig.add_subplot(212)
df.plot(x='hour', y= 'counts',kind='bar',ax=ax2)
plt.tight_layout()
plt.savefig("weekly_heat.jpg")
plt.show()

In [ ]:
chat['text_len'] = chat['text'].map(lambda x: len(x))

def get_stats(group):
    return {'max': group.max(), 'count': group.count(), 'sum': group.sum(),'mean':group.mean()}

df=pd.DataFrame(chat['text_len'].groupby(chat['sender']).apply(get_stats).unstack())
df=df.reset_index()

In [ ]:
fig = plt.figure(figsize=(20,15))

ax = fig.add_subplot(411)
ax2 = fig.add_subplot(412)
ax3 = fig.add_subplot(413)
ax4 = fig.add_subplot(414)
ax.xaxis.set_visible(False)
ax2.xaxis.set_visible(False)
ax3.xaxis.set_visible(False)

ax.title.set_size(30)
for label in (ax4.get_xticklabels()+ax.get_yticklabels()+ax2.get_yticklabels()+ax3.get_yticklabels()+ax4.get_yticklabels()):
    label.set_fontname('Arial')
    label.set_fontsize(15)
df['count'].plot(kind='bar', color='red', ax=ax,title='number of messages') #, width=0.25)
df['max'].plot(kind='bar', color='blue', ax=ax2,title='max message length')#, width=0.25)
df['mean'].plot(kind='bar', color='green', ax=ax3,title='average message length')#, width=0.25)
df['sum'].plot(kind='bar', color='yellow', ax=ax4,title='total charactes typed')#, width=0.25)
ax.title.set_size(20)
ax2.title.set_size(20)
ax3.title.set_size(20)
ax4.title.set_size(20)
ax4.set_xticklabels(df.sender[:],ha='right')
plt.savefig('stats.jpg')
plt.show()

In [ ]:
flat_list ='' 
for item in chat.loc[chat.text != '<Media omitted>\n'].text:
    if not isinstance(item,list):
        flat_list=flat_list+item
words=tokenizer.tokenize(flat_list)
count=set(words)
dic=nltk.FreqDist(words)
fig = plt.figure(figsize=(20,10))
ax=fig.add_subplot(111)
for label in (ax.get_xticklabels()+ax.get_yticklabels()):
    label.set_fontname('Arial')
    label.set_fontsize(15)
nltk.FreqDist(words).plot(50, cumulative=False)
fig.savefig('word_count.jpg')

In [ ]:
# lower max_font_size
wordcloud = WordCloud(max_font_size=60,background_color="white").generate(flat_list)
fig = plt.figure(figsize=(10,10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()
fig.savefig('word_cloud.jpg')

In [ ]:
emoticons = set(range(int('1f600',16), int('1f650', 16)))
emojis=''
for x in flat_list:
    if ord(x) in emoticons:
        emojis=emojis+x

normal_word = r"(?:\w[\w']+)"
# 2+ consecutive punctuations, e.x. :)
ascii_art = r"(?:[{punctuation}][{punctuation}]+)".format(punctuation=string.punctuation)
# a single character that is not alpha_numeric or other ascii printable
emoji = r"(?:[^\s])(?<![\w{ascii_printable}])".format(ascii_printable=string.printable)
regexp = r"{normal_word}|{ascii_art}|{emoji}".format(normal_word=normal_word, ascii_art=ascii_art,
                                                     emoji=emoji)
#emojis
# Generate a word cloud image
# The Symbola font includes most emoji
wordcloud = WordCloud(font_path='Symbola.ttf',background_color="white", regexp=regexp).generate(emojis)
fig = plt.figure(figsize=(10,10))
# Display the generated image:
# the matplotlib way:
plt.imshow(wordcloud)
plt.title("most_common_smileys")
plt.axis("off")
plt.savefig('emojis.jpg')
plt.show()

In [ ]:
# for i in chat.sender.unique():
#     flat_list ='' 
#     for item in (chat.loc[(chat['sender'] == str(i)) & (chat.text != '<Media omitted>\n')]['text']):
#         if not isinstance(item,list):
#             flat_list=flat_list+item
#     allWords = nltk.tokenize.word_tokenize(flat_list)
#     allWordDist = nltk.FreqDist(w.lower() for w in allWords)

#     stopwords = nltk.corpus.stopwords.words('english')
#     allWordDist = nltk.FreqDist(w.lower() for w in allWords if w not in stopwords)  
#     mostCommon= allWordDist.most_common(10)
#     print(i, mostCommon)

In [ ]:
fig = plt.figure(figsize=(20,20))
cols=5
rows= int(len(chat.sender.unique())/5)+1
plot_num=1
for indx, item in enumerate(chat.sender.unique()):
    #print (item)
    flat_list ='' 
    for x in (chat.loc[(chat['sender'] == str(item)) & (chat.text != '<Media omitted>\n')]['text']):
        if not isinstance(x,list):
            flat_list=flat_list+x
    if len(flat_list)>50:
        wordcloud = WordCloud(max_font_size=60,background_color="white").generate(flat_list)
        plt.subplot(rows, cols, plot_num)
        plot_num=plot_num+1
        plt.imshow(wordcloud, interpolation="bilinear")
        plt.title(str(item))
        plt.axis("off")
plt.tight_layout()
#plt.title("Word_Colud of Members")
plt.savefig("all.jpg")
plt.show()
#

In [ ]:
for indx, item in enumerate(chat.sender.unique()):
    df=pd.DataFrame(chat.loc[(chat['sender'] == str(item))].groupby(['Weekday','hour']).size().rename('counts'))
    df=df.reset_index()
    gridkw = dict(height_ratios=[4, 1])
    fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw=gridkw,figsize=(20,15)) 
    #plt.yticks(F6.Weekday, DaysofWeeks)
    result = df.pivot(index='Weekday', columns='hour', values='counts')
    sns.heatmap(result, annot=True,yticklabels=DaysofWeeks, fmt="g", ax=ax1, cbar_kws={"orientation": "horizontal"})
    ax1.set_title("weekly heatmap_"+str(item))
    df=pd.DataFrame(chat.groupby(['hour']).size().rename('counts'))
    df=df.reset_index()
    #ax2=fig.add_subplot(212)
    df.plot(x='hour', y= 'counts',kind='bar',ax=ax2)
    plt.tight_layout()
    
    plt.savefig("personalheat/weekly_heat_"+str(item)+".jpg")
    plt.show()